#  Data Prep & Filtering

In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from pandas import Series, DataFrame
import time
import turicreate as tc
from sklearn.model_selection import train_test_split
from functools import reduce
import random

In [3]:

file1 = "Data1.csv"
file2 = "Data2.csv"


In [7]:
def pre_process_df(filename):
    if filename:
        df = pd.read_csv(filename)
        df = df.drop_duplicates()
        df = df[df.brand.notnull()]
        return df[df.price > 0]
    return None
  
def gen_purchase_count(files):
    df = pd.concat([pre_process_df(f) for f in files])
    df = df.loc[:, ['event_type', 'product_id', 'user_id']]
    df['purchase_count'] = [1 if e == 'purchase' else 0 for e in df['event_type']]
    df = df.drop(['event_type'], axis = 1)
    df = df.groupby(['product_id', 'user_id']).sum()
    df = df[df.purchase_count != 0]
    return df.reset_index()

In [8]:
purchase_count = gen_purchase_count([file1,file2])

In [9]:
purchase_count

,product_id,user_id,purchase_count
0,3762,254751820,1
1,3762,258117654,1
2,3762,288191779,1
3,3762,293416727,1
4,3762,296682241,1
...,...,...,...
318443,5909237,571392419,1
318444,5909237,579752599,1
318445,5909238,579137281,1
318446,5909240,579137281,1


In [10]:
user_id  = np.unique(purchase_count['user_id'])
num_products_purchased = np.zeros(len(user_id))
for loc, user in enumerate(user_id):
    num_products_purchased[loc] = np.sum(purchase_count['user_id'] == user)

In [7]:
num_products_purchased

array([2., 2., 3., ..., 2., 1., 1.])

In [8]:
np.sum(purchase_count['user_id'] == 579681799)

3

In [9]:
purchase_count[purchase_count['user_id'] == 579681799]

,brand,user_id,purchase_count
72519,irisk,579681799,1
89748,lovely,579681799,2
151239,zinger,579681799,10


In [10]:
user_id

array([  9794320,  10079204,  10280338, ..., 579924768, 579925377,
       579944216])

In [11]:
idx = np.where(num_products_purchased>=10)

In [13]:
filitered_user_id = user_id[idx]

In [14]:
filtered_data = purchase_count.loc[purchase_count['user_id'].isin(filitered_user_id)]

In [15]:
filtered_data

,product_id,user_id,purchase_count
1,3762,258117654,1
2,3762,288191779,1
12,3762,419505462,1
19,3762,445330308,1
23,3762,469218581,1
...,...,...,...
318442,5909233,502873541,1
318443,5909237,571392419,1
318445,5909238,579137281,1
318446,5909240,579137281,1


In [16]:
filtered_data=filtered_data[['user_id','product_id','purchase_count']]

In [17]:
filtered_data.columns=['customerId','productId','purchase_count']

In [18]:
data=filtered_data

In [19]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy
data_dummy = create_data_dummy(data)

In [20]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')

In [21]:
df_matrix

productId,3762,3763,3774,3776,3806,3928,3929,3936,3945,3959,...,5908835,5909042,5909057,5909070,5909231,5909233,5909237,5909238,5909240,5909245
customerId,,,,,,,,,,,,,,,,,,,,,
10280338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29025780,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36180886,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40821287,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43713532,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579811230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
579811378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
579849574,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())

In [23]:
d = df_matrix_norm.reset_index() 
d.index.names = ['scaled_purchase_freq'] 
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()


In [24]:
print(data_norm.shape)
data_norm.head()

(111468, 3)


,customerId,productId,scaled_purchase_freq
357,258117654,3762,0.0
483,288191779,3762,0.0
1745,419505462,3762,0.0
2362,445330308,3762,0.0
2978,469218581,3762,0.0


In [25]:
def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [26]:
train_data, test_data = split_data(data)
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

In [27]:
user_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(filtered_data[user_id])
n_rec = 5 # number of items to recommend
n_display = 30 # to display the first few rows in an output dataset

In [28]:
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
            model = tc.item_similarity_recommender.create(train_data, 
                                                        user_id=user_id, 
                                                        item_id=item_id, 
                                                        target=target, 
                                                    similarity_type='pearson')
    elif name == 'jaccard':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='jaccard')

    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model


In [29]:
name = 'popularity'
target = 'purchase_count'
popularity = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 144838 observations with 9672 users and 15007 items.

Data prepared in: 1.92229s

144838 observations to process; with 15007 unique items.

recommendations finished on 1000/181048 queries. users per second: 15830.8

...

recommendations finished on 178000/181048 queries. users per second: 21757.8

recommendations finished on 179000/181048 queries. users per second: 21739.4

recommendations finished on 180000/181048 queries. users per second: 21662.5

recommendations finished on 181000/181048 queries. users per second: 21605

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
| 258117654  |  5797165  |  3.0  |  1   |
| 258117654  |   20921   |  3.0  |  2   |
| 258117654  |  5888538  |  3.0  |  3   |
| 258117654  |  5769287  |  3.0  |  4   |
| 258117654  |  5805502  |  3.0  |  5   |
| 288191779  |  5797165  |  3.0  |  1   |
| 288191779  |   20921   |  3.0  |  2   |
| 288191779  |  5888538  |  3.0  |  3   |
| 288191779  |  5769287  |  3.0  |  4   |
| 288191779  |  5805502  |  3.0  |  5   |
| 419505462  |  5797165  |  3.0  |  1   |
| 419505462  |   20921   |  3.0  |  2   |
| 419505462  |  5888538  |  3.0  |  3   |
| 419505462  |  5769287  |  3.0  |  4   |
| 419505462  |  5805502  |  3.0  |  5   |
| 445330308  |  5797165  |  3.0  |  1   |
| 445330308  |   20921   |  3.0  |  2   |
| 445330308  |  5888538  |  3.0  |  3   |
| 445330308  |  5769287  |  3.0  |  4   |
| 445330308  |  5805502  |  3.0  |  5   |
| 469218581  |  5797165  |  3.0  |

In [30]:
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 144838 observations with 9672 users and 15008 items.

Data prepared in: 0.149638s

...

recommendations finished on 178000/181048 queries. users per second: 21323.6

recommendations finished on 179000/181048 queries. users per second: 21320

recommendations finished on 180000/181048 queries. users per second: 21316.8

recommendations finished on 181000/181048 queries. users per second: 21268.6

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
| 258117654  |  5622676  |  1.0  |  1   |
| 258117654  |  5890938  |  1.0  |  2   |
| 258117654  |  5674915  |  1.0  |  3   |
| 258117654  |  5754292  |  1.0  |  4   |
| 258117654  |  5692280  |  1.0  |  5   |
| 288191779  |  5622676  |  1.0  |  1   |
| 288191779  |  5890938  |  1.0  |  2   |
| 288191779  |  5674915  |  1.0  |  3   |
| 288191779  |  5754292  |  1.0  |  4   |
| 288191779  |  5692280  |  1.0  |  5   |
| 419505462  |  5622676  |  1.0  |  1   |
| 419505462  |  5890938  |  1.0  |  2   |
| 419505462  |  5674915  |  1.0  |  3   |
| 419505462  |  5754292  |  1.0  |  4   |
| 419505462  |  5692280  |  1.0  |  5   |
| 445330308  |  5622676  |  1.0  |  1   |
| 445330308  |  5890938  |  1.0  |  2   |
| 445330308  |  5674915  |  1.0  |  3   |
| 445330308  |  5754292  |  1.0  |  4   |
| 445330308  |  5692280  |  1.0  |  5   |
| 469218581  |  5622676  |  1.0  |

In [31]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 89174 observations with 9612 users and 3183 items.

Data prepared in: 0.071275s

89174 observations to process; with 3183 unique items.

recommendations finished on 1000/181048 queries. users per second: 68785.3

...

recommendations finished on 177000/181048 queries. users per second: 90412.5

recommendations finished on 178000/181048 queries. users per second: 90413.8

recommendations finished on 179000/181048 queries. users per second: 90301.9

recommendations finished on 180000/181048 queries. users per second: 90054.1

recommendations finished on 181000/181048 queries. users per second: 89655.6

In [42]:
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 144838 observations with 9672 users and 15007 items.

Data prepared in: 0.111562s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 178000/181048 queries. users per second: 16468

recommendations finished on 179000/181048 queries. users per second: 16478.5

recommendations finished on 180000/181048 queries. users per second: 16481.5

recommendations finished on 181000/181048 queries. users per second: 16451.2

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
| 258117654  |  5804264  | 0.06981423497200012  |  1   |
| 258117654  |    5256   | 0.06981423497200012  |  2   |
| 258117654  |  5809146  | 0.057925575971603395 |  3   |
| 258117654  |  5804296  | 0.055694013833999634 |  4   |
| 258117654  |  5788729  | 0.055694013833999634 |  5   |
| 288191779  |  5645615  | 0.08520509799321492  |  1   |
| 288191779  |    5067   |  0.0832304060459137  |  2   |
| 288191779  |  5804264  | 0.08302372694015503  |  3   |
| 288191779  |    5256   | 0.08302372694015503  |  4   |
| 288191779  |  5645620  | 0.07210172712802887  |  5   |
| 419505462  |    4768   | 0.043328512992177694 |  1   |
| 419505462  |    4554   | 0.03367892120565687  |  2   |
| 419505462  |    4765   | 0.031920403242111206 |  3   |
| 419505462  |    4649   | 0.028358167835644314 |  4   |
| 419505462  |    4766   | 0.02

In [32]:
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 144838 observations with 9672 users and 15008 items.

Data prepared in: 0.179677s

...

recommendations finished on 178000/181048 queries. users per second: 16724.4

recommendations finished on 179000/181048 queries. users per second: 16720.1

recommendations finished on 180000/181048 queries. users per second: 16700.6

recommendations finished on 181000/181048 queries. users per second: 16641.2

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
| 258117654  |  5769986  | 0.07162437174055311  |  1   |
| 258117654  |  5805750  | 0.06902786758210924  |  2   |
| 258117654  |  5835177  | 0.06353014707565308  |  3   |
| 258117654  |  5712788  | 0.05823798312081231  |  4   |
| 258117654  |  5769981  | 0.05731268061531915  |  5   |
| 288191779  |  5663277  | 0.11693533108784603  |  1   |
| 288191779  |    4092   | 0.11693533108784603  |  2   |
| 288191779  |    4131   | 0.08268576860427856  |  3   |
| 288191779  |  5764550  | 0.08020166250375602  |  4   |
| 288191779  |    3959   |  0.0732483772131113  |  5   |
| 419505462  |  5798778  | 0.03443503996421551  |  1   |
| 419505462  |  5700696  | 0.03443503996421551  |  2   |
| 419505462  |  5870817  | 0.031920126799879406 |  3   |
| 419505462  |  5770320  | 0.031920126799879406 |  4   |
| 419505462  |  5848403  | 0.03

In [33]:
name = 'cosine' 
target = 'scaled_purchase_freq' 
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 89174 observations with 9612 users and 3183 items.

Data prepared in: 0.06463s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 177000/181048 queries. users per second: 66894.5

recommendations finished on 178000/181048 queries. users per second: 66994.1

recommendations finished on 179000/181048 queries. users per second: 67004.4

recommendations finished on 180000/181048 queries. users per second: 66985.5

recommendations finished on 181000/181048 queries. users per second: 66391

In [35]:
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 144838 observations with 9672 users and 15007 items.

Data prepared in: 0.095831s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 178000/181048 queries. users per second: 14709.7

recommendations finished on 179000/181048 queries. users per second: 14699.9

recommendations finished on 180000/181048 queries. users per second: 14693.1

recommendations finished on 181000/181048 queries. users per second: 14651.5

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
| 258117654  |  5797165  |  3.0  |  1   |
| 258117654  |   20921   |  3.0  |  2   |
| 258117654  |  5888538  |  3.0  |  3   |
| 258117654  |  5805502  |  3.0  |  4   |
| 258117654  |  5769287  |  3.0  |  5   |
| 288191779  |  5797165  |  3.0  |  1   |
| 288191779  |   20921   |  3.0  |  2   |
| 288191779  |  5888538  |  3.0  |  3   |
| 288191779  |  5805502  |  3.0  |  4   |
| 288191779  |  5769287  |  3.0  |  5   |
| 419505462  |  5797165  |  3.0  |  1   |
| 419505462  |   20921   |  3.0  |  2   |
| 419505462  |  5888538  |  3.0  |  3   |
| 419505462  |  5805502  |  3.0  |  4   |
| 419505462  |  5769287  |  3.0  |  5   |
| 445330308  |  5797165  |  3.0  |  1   |
| 445330308  |   20921   |  3.0  |  2   |
| 445330308  |  5888538  |  3.0  |  3   |
| 445330308  |  5805502  |  3.0  |  4   |
| 445330308  |  5769287  |  3.0  |  5   |
| 469218581  |  5797165  |  3.0  |

In [36]:
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 144838 observations with 9672 users and 15008 items.

Data prepared in: 0.102204s

...

recommendations finished on 178000/181048 queries. users per second: 10330.1

recommendations finished on 179000/181048 queries. users per second: 10330.7

recommendations finished on 180000/181048 queries. users per second: 10325.3

recommendations finished on 181000/181048 queries. users per second: 10321

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
| 258117654  |  5622676  |  0.0  |  1   |
| 258117654  |  5890938  |  0.0  |  2   |
| 258117654  |  5674915  |  0.0  |  3   |
| 258117654  |  5754292  |  0.0  |  4   |
| 258117654  |  5692280  |  0.0  |  5   |
| 288191779  |  5622676  |  0.0  |  1   |
| 288191779  |  5890938  |  0.0  |  2   |
| 288191779  |  5674915  |  0.0  |  3   |
| 288191779  |  5754292  |  0.0  |  4   |
| 288191779  |  5692280  |  0.0  |  5   |
| 419505462  |  5622676  |  0.0  |  1   |
| 419505462  |  5890938  |  0.0  |  2   |
| 419505462  |  5674915  |  0.0  |  3   |
| 419505462  |  5754292  |  0.0  |  4   |
| 419505462  |  5692280  |  0.0  |  5   |
| 445330308  |  5622676  |  0.0  |  1   |
| 445330308  |  5890938  |  0.0  |  2   |
| 445330308  |  5674915  |  0.0  |  3   |
| 445330308  |  5754292  |  0.0  |  4   |
| 445330308  |  5692280  |  0.0  |  5   |
| 469218581  |  5622676  |  0.0  |

In [37]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 89174 observations with 9612 users and 3183 items.

Data prepared in: 0.096446s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 177000/181048 queries. users per second: 48070

recommendations finished on 178000/181048 queries. users per second: 48116

recommendations finished on 179000/181048 queries. users per second: 48161.9

recommendations finished on 180000/181048 queries. users per second: 48154

recommendations finished on 181000/181048 queries. users per second: 47956.2

In [38]:
name = 'jaccard'
target = 'purchase_count'
jacc = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 144838 observations with 9672 users and 15007 items.

Data prepared in: 0.102781s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 178000/181048 queries. users per second: 16735.6

recommendations finished on 179000/181048 queries. users per second: 16739.2

recommendations finished on 180000/181048 queries. users per second: 16736

recommendations finished on 181000/181048 queries. users per second: 16712.3

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
| 258117654  |  5614842  |       0.034375       |  1   |
| 258117654  |  5749227  | 0.023593074083328246 |  2   |
| 258117654  |  5769986  | 0.02254902124404907  |  3   |
| 258117654  |  5597372  | 0.022134387493133546 |  4   |
| 258117654  |  5805750  | 0.021896547079086302 |  5   |
| 288191779  |  5803428  | 0.041666666666666664 |  1   |
| 288191779  |  5828847  | 0.041666666666666664 |  2   |
| 288191779  |  5862931  | 0.041666666666666664 |  3   |
| 288191779  |  5645615  | 0.04069548348585764  |  4   |
| 288191779  |  5848331  | 0.03333333134651184  |  5   |
| 419505462  |    4768   | 0.024493466530527388 |  1   |
| 419505462  |    4554   | 0.015869357756205967 |  2   |
| 419505462  |  5772461  | 0.014919191598892212 |  3   |
| 419505462  |    4765   | 0.012645419154848372 |  4   |
| 419505462  |    4571   | 0.01

In [39]:
name = 'jaccard'
target = 'purchase_dummy'
jacc_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 144838 observations with 9672 users and 15008 items.

Data prepared in: 0.121184s

...

recommendations finished on 178000/181048 queries. users per second: 15358

recommendations finished on 179000/181048 queries. users per second: 15359.4

recommendations finished on 180000/181048 queries. users per second: 15351.9

recommendations finished on 181000/181048 queries. users per second: 15309.1

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
| 258117654  |  5769986  | 0.03901234600279066  |  1   |
| 258117654  |  5614842  | 0.036150978671179876 |  2   |
| 258117654  |  5769981  | 0.028730160660213895 |  3   |
| 258117654  |  5805750  | 0.02525252103805542  |  4   |
| 258117654  |    3945   | 0.023073878553178575 |  5   |
| 288191779  |  5663277  | 0.06837606888550979  |  1   |
| 288191779  |    4092   | 0.06837606888550979  |  2   |
| 288191779  |    4131   |  0.0489203746502216  |  3   |
| 288191779  |  5764550  |  0.0415384631890517  |  4   |
| 288191779  |  5812621  | 0.038461538461538464 |  5   |
| 419505462  |  5772461  | 0.017718588483744656 |  1   |
| 419505462  |  5772471  | 0.01722345886559322  |  2   |
| 419505462  |  5809875  | 0.016608731500033676 |  3   |
| 419505462  |    4600   | 0.016348507897607212 |  4   |
| 419505462  |  5772307  | 0.01

In [40]:
name = 'jaccard'
target = 'scaled_purchase_freq'
jacc_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 89174 observations with 9612 users and 3183 items.

Data prepared in: 0.120997s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 177000/181048 queries. users per second: 55831.8

recommendations finished on 178000/181048 queries. users per second: 55811

recommendations finished on 179000/181048 queries. users per second: 55774.4

recommendations finished on 180000/181048 queries. users per second: 55753.7

recommendations finished on 181000/181048 queries. users per second: 55564

In [43]:
models_w_counts = [popularity, cos, pear,jacc]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy,jacc_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm,jacc_norm]
names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts','Jaccard Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy','Jaccard Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts','Jaccard Similarity on Scaled Purchase Counts']

In [44]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/9229 queries. users per second: 14708.5

recommendations finished on 2000/9229 queries. users per second: 15165.3

recommendations finished on 3000/9229 queries. users per second: 14991.4

recommendations finished on 4000/9229 queries. users per second: 14391.2

...

recommendations finished on 5000/8186 queries. users per second: 68225.9

recommendations finished on 6000/8186 queries. users per second: 65553

recommendations finished on 7000/8186 queries. users per second: 65294.2

recommendations finished on 8000/8186 queries. users per second: 65285.3


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    | 0.0054971903249450265 | 0.0016655049510642269 |
|   2    |  0.004275592474957246 | 0.0026604814163558627 |
|   3    |  0.003664793549963349 | 0.0033569376191643755 |
|   4    | 0.0031150745174688492 |  0.004002348483241255 |
|   5    | 0.0028585389689714104 |  0.004597930318134109 |
|   6    |  0.002646795341640203 |  0.005375002283820787 |
|   7    |  0.00246064709783253  |  0.005752739501356224 |
|   8    |  0.002336305888101638 |  0.006064118304977886 |
|   9    | 0.0022938892960881687 |  0.006589771154720638 |
|   10   | 0.0022843879794771566 |  0.007532582923943355 |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.21710168707975092

Per User RMSE (best)
+------------+------+-------+
| customerId 

In [45]:
eval_counts

[{'precision_recall_by_user': Columns:
  	customerId	int
  	cutoff	int
  	precision	float
  	recall	float
  	count	int
  
  Rows: 166122
  
  Data:
  +------------+--------+-----------+--------+-------+
  | customerId | cutoff | precision | recall | count |
  +------------+--------+-----------+--------+-------+
  | 497361708  |   1    |    0.0    |  0.0   |   5   |
  | 497361708  |   2    |    0.0    |  0.0   |   5   |
  | 497361708  |   3    |    0.0    |  0.0   |   5   |
  | 497361708  |   4    |    0.0    |  0.0   |   5   |
  | 497361708  |   5    |    0.0    |  0.0   |   5   |
  | 497361708  |   6    |    0.0    |  0.0   |   5   |
  | 497361708  |   7    |    0.0    |  0.0   |   5   |
  | 497361708  |   8    |    0.0    |  0.0   |   5   |
  | 497361708  |   9    |    0.0    |  0.0   |   5   |
  | 497361708  |   10   |    0.0    |  0.0   |   5   |
  +------------+--------+-----------+--------+-------+
  [166122 rows x 5 columns]
  Note: Only the head of the SFrame is printed.
  You 

## Jaccrd Similarity here is the best Similarity

In [46]:
final_model = tc.item_similarity_recommender.create(tc.SFrame(data_dummy), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_dummy', similarity_type='jaccard')
recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 181048 observations with 9672 users and 15813 items.

Data prepared in: 0.147732s

...

recommendations finished on 178000/181048 queries. users per second: 15964.6

recommendations finished on 179000/181048 queries. users per second: 15959.6

recommendations finished on 180000/181048 queries. users per second: 15922.2

recommendations finished on 181000/181048 queries. users per second: 15777.6

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
| 258117654  |  5769986  | 0.037832189689983024 |  1   |
| 258117654  |  5614842  | 0.03466154770417647  |  2   |
| 258117654  |  5769981  | 0.027372631159695713 |  3   |
| 258117654  |  5805750  | 0.024969187649813564 |  4   |
| 258117654  |    3928   | 0.024153075434944847 |  5   |
| 288191779  |    4092   | 0.04340721579159007  |  1   |
| 288191779  |  5788728  | 0.04075630272136015  |  2   |
| 288191779  |  5614841  | 0.04020581525914809  |  3   |
| 288191779  |    4131   | 0.038680925088770246 |  4   |
| 288191779  |  5769982  |  0.0372549014932969  |  5   |
| 419505462  |    4768   | 0.02761567792584819  |  1   |
| 419505462  |    4554   | 0.021009076026178176 |  2   |
| 419505462  |  5772461  | 0.02028870198034471  |  3   |
| 419505462  |  5772471  | 0.01865997814363049  |  4   |
| 419505462  |  5772307  | 0.01

In [47]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head()

(905240, 4)


,customerId,productId,score,rank
0,258117654,5769986,0.037832,1
1,258117654,5614842,0.034662,2
2,258117654,5769981,0.027373,3
3,258117654,5805750,0.024969,4
4,258117654,3928,0.024153,5


In [ ]:
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=5)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: ' , '.join(x.astype(str)))
    df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates() \
        .sort_values('customerId').set_index('customerId')

In [48]:
recomendation = final_model.recommend(users=users_to_recommend, k=5)
df_rec = recomendation.to_dataframe()
df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
    .transform(lambda x: ' , '.join(x.astype(str)))
df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates() \
    .sort_values('customerId').set_index('customerId')

recommendations finished on 1000/181048 queries. users per second: 15526.5

recommendations finished on 2000/181048 queries. users per second: 16736.1

recommendations finished on 3000/181048 queries. users per second: 16971.4

recommendations finished on 4000/181048 queries. users per second: 16392.2

recommendations finished on 5000/181048 queries. users per second: 16164.1

...

recommendations finished on 177000/181048 queries. users per second: 15023

recommendations finished on 178000/181048 queries. users per second: 15032.1

recommendations finished on 179000/181048 queries. users per second: 15043

recommendations finished on 180000/181048 queries. users per second: 15042.2

recommendations finished on 181000/181048 queries. users per second: 14942

In [49]:
df_output=df_output.reset_index()

In [50]:
df_output.head()

,customerId,recommendedProducts
0,10280338,"5816170 , 5816178 , 5816172 , 5816169 , 585062..."
1,29025780,"5815567 , 5815569 , 5818266 , 5818264 , 581826..."
2,36180886,"5819234 , 5819244 , 5819223 , 5819222 , 580783..."
3,40821287,"5304 , 5700046 , 5810157 , 4958 , 5848387 , 53..."
4,43713532,"5833323 , 5848909 , 5772308 , 5833330 , 580929..."


In [57]:
df_output.shape

(9672, 2)

## This is for streamlit 

In [51]:
# new=recomendation.to_dataframe()

# new.columns

# new.columns=['user_id','product_id','score', 'rank']

# new1=new[["user_id","product_id","rank"]]

# new1.to_csv('new_product.csv',index=False)

In [61]:
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates() \
        .sort_values('customerId').set_index('customerId')
    if print_csv:
        df_output.to_csv('option1_recommendation.csv')
        print("An output file can be found in 'output' folder with name 'option1_recommendation.csv'")
    return df_output

In [62]:
df_output = create_output(pear_norm, users_to_recommend, 5, print_csv=True)
print(df_output.shape)
df_output.head()

recommendations finished on 1000/181048 queries. users per second: 26219.9

recommendations finished on 2000/181048 queries. users per second: 37908.9

recommendations finished on 3000/181048 queries. users per second: 38741.7

recommendations finished on 4000/181048 queries. users per second: 43552.1

recommendations finished on 5000/181048 queries. users per second: 47285.3

...

recommendations finished on 179000/181048 queries. users per second: 57735.1

recommendations finished on 180000/181048 queries. users per second: 57702.4

recommendations finished on 181000/181048 queries. users per second: 57401.7

An output file can be found in 'output' folder with name 'option1_recommendation.csv'
(9672, 1)


,recommendedProducts
customerId,
10280338,5887685|5812505|5888344|5798771|5648651|588768...
29025780,5887685|5812505|5888344|5798771|5648651|588768...
36180886,5887685|5812505|5888344|5798771|5648651|588768...
40821287,5887685|5812505|5888344|5798771|5648651|588768...
43713532,5887685|5812505|5888344|5798771|5648651|588768...


In [63]:
def customer_recomendation(customer_id):
    if customer_id not in df_output.index:
        print('Customer not found.')
        return customer_id
    return df_output.loc[customer_id]

In [64]:
customer_recomendation(40821287)

recommendedProducts    5887685|5812505|5888344|5798771|5648651|588768...
Name: 40821287, dtype: object